In [ ]:
import pyvisa
import time
import matplotlib.pyplot as plt
rm = pyvisa.ResourceManager()
visa_list = rm.list_resources()
print(visa_list)

In [ ]:
k2636a = rm.open_resource('GPIB0::26::INSTR')
tc = rm.open_resource('GPIB0::4::INSTR')
mag = rm.open_resource('GPIB0::3::INSTR')

In [ ]:
float(tc.query('KRDG? A'))

In [ ]:
"""
k2636a.write("smua.source.func = smua.OUTPUT_DCAMPS")
k2636a.write("smua.source.limitv = 5")
k2636a.write("smua.source.leveli = 100e-6")
k2636a.write("smua.source.autorangei = smua.AUTORANGE_ON")
k2636a.write("smub.measure.rangev = 2")
"""

In [ ]:
k2636a.write("smub.measure.rangev = 0.2")
k2636a.write("smub.source.func = smub.OUTPUT_DCAMPS")
k2636a.write("smub.source.rangei = 1e-9")
k2636a.write("smub.source.leveli = 0")
k2636a.write("smub.measure.nplc = 3")

In [ ]:
k2636a.write("smua.measure.rangev = 0.2")
k2636a.write("smua.source.func = smub.OUTPUT_DCAMPS")

# measure V
k2636a.write("smua.source.rangei = 1e-9")
k2636a.write("smua.source.leveli = 0")

k2636a.write("smub.measure.nplc = 1")

In [ ]:
k2636a.write("smua.source.output = smua.OUTPUT_ON")
k2636a.write("smub.source.output = smub.OUTPUT_ON")

In [ ]:
import numpy as np

In [ ]:
with open("rt.txt","w") as f:
    f.write("Temperature (K), B_Voltage (V), A_Voltage (V)\n")

In [ ]:
k2636a.write("smua.source.output = smua.OUTPUT_OFF")
k2636a.write("smub.source.output = smub.OUTPUT_OFF")

In [ ]:
k2636a.write("smua.source.output = smua.OUTPUT_ON")
time.sleep(2)
k2636a.write("smub.source.output = smub.OUTPUT_ON")
while True:
    i = 1
    t = float(tc.query('KRDG? A'))
    vb = float(k2636a.query("print(smub.measure.v())"))
    va = float(k2636a.query("print(smua.measure.v())"))
    with open("rt.txt","a") as f:
        f.write(f"{t}, {vb}, {va}\n")
    time.sleep(20)
    if t <5.:
        break

In [ ]:
import numpy as np

In [ ]:
h = np.arange(10, -10, -0.2)
hl = np.concatenate((h, -h), axis=0)
hi = (h * 3200).round().astype(np.int16)
hil= np.concatenate((hi, -hi), axis=0)
print(hil.shape)

In [ ]:
k2636a.write("smub.reset()")
k2636a.write("smub.source.func = smub.OUTPUT_DCAMPS")
k2636a.write("smub.source.leveli = 0")
k2636a.write("smub.measure.autorangev = smub.AUTORANGE_ON")

In [ ]:
# 2-wire + 4-wire MR or HR measurement
with open("mr.txt","w") as f:
    f.write("magnetic field(kG), B_Voltage (V), A_Voltage (V)\n")
mag.write("A1D32000")
k2636a.write("smua.source.output = smua.OUTPUT_ON")
k2636a.write("smub.source.output = smub.OUTPUT_ON")
time.sleep(5)
for h, i in zip(hil, hl):
    mag.write("A1D" + str(h))
    time.sleep(1.5)
    vb = float(k2636a.query("print(smub.measure.v())"))
    va = float(k2636a.query("print(smua.measure.v())"))
    with open("mr.txt","a") as f:
        f.write(f"{i}, {vb}, {va}\n")
mag.write("A1D0")
k2636a.write("smua.source.output = smua.OUTPUT_OFF")
k2636a.write("smub.source.output = smub.OUTPUT_OFF")

In [ ]:
# MR and HR measurement
filename = "H1668_outofplane_50ua_171K.txt"
with open(filename,"w") as f:
    f.write("magnetic field(kG), MR (V), HR (V)\n")
mag.write("A1D32000")
k2636a.write("smua.source.output = smua.OUTPUT_ON")
k2636a.write("smub.source.output = smub.OUTPUT_ON")
time.sleep(5)
for h, i in zip(hil, hl):
    mag.write("A1D" + str(h))
    time.sleep(1)
    vb = float(k2636a.query("print(smub.measure.v())"))
    va = float(k2636a.query("print(smua.measure.v())"))
    with open(filename,"a") as f:
        f.write(f"{i}, {vb}, {va}\n")
mag.write("A1D0")
k2636a.write("smua.source.output = smua.OUTPUT_OFF")
k2636a.write("smub.source.output = smub.OUTPUT_OFF")

In [ ]:
filename = "H1668_outofplane_40ua.txt"
mr = np.loadtxt(filename, delimiter=",", skiprows=1).T
plt.plot(mr[0], mr[2])
plt.show()